# New Section

In [1]:

import sys
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 2000)

import sklearn.model_selection
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import datetime

import joblib
import pickle
import tempfile


import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


# S3 bucket name
bucket = 'sagemaker-us-west-2-479862333671'
subfolder = ''
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']


In [2]:
sales_train_evaluation_lag_rolling = pd.read_csv('s3://{}/{}'.format(bucket, 'data/sales_train_evaluation_lag_rolling.csv'))

In [3]:
sales_train_evaluation_lag_rolling

,demand,item_id,dept_id,cat_id,store_id,state_id,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,date_day,date_week,date_month,date_year,date_week_m,lag_28,lag_35,lag_42,lag_49,lag_56,rolling_median_7,rolling_median_14,rolling_median_30,rolling_median_60,rolling_median_180
0,0,1446,3,1,9,2,19,30,4,4,2,0,0,0,3.17,16,7,2,0,3,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.5,0.0,0.0
1,1,1446,3,1,9,2,20,30,4,4,2,0,0,0,3.17,17,7,2,0,3,2.0,2.0,0.0,0.0,0.0,1.0,0.5,0.5,0.0,0.0
2,3,1446,3,1,9,2,21,30,4,4,2,0,0,0,3.17,18,7,2,0,3,0.0,0.0,0.0,1.0,2.0,1.0,0.5,0.0,0.0,0.0
3,0,1458,3,1,9,2,19,30,4,4,2,0,0,0,7.33,16,7,2,0,3,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
4,0,1458,3,1,9,2,20,30,4,4,2,0,0,0,7.33,17,7,2,0,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46220039,1,1432,2,0,9,2,1940,30,4,4,2,0,0,0,2.98,21,20,5,5,3,0.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0
46220040,1,1433,2,0,9,2,1940,30,4,4,2,0,0,0,2.48,21,20,5,5,3,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
46220041,0,1434,2,0,9,2,1940,30,4,4,2,0,0,0,3.98,21,20,5,5,3,5.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0
46220042,1,1435,2,0,9,2,1940,30,4,4,2,0,0,0,1.28,21,20,5,5,3,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0


In [2]:
#train = sales_train_evaluation_lag_rolling[sales_train_evaluation_lag_rolling['d'] < 1914]
train = pd.read_csv('s3://{}/{}'.format(bucket, 'data/train.csv'))

In [3]:
train

,demand,item_id,dept_id,cat_id,store_id,state_id,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,date_day,date_week,date_month,date_year,date_week_m,lag_28,lag_35,lag_42,lag_49,lag_56,rolling_median_7,rolling_median_14,rolling_median_30,rolling_median_60,rolling_median_180
0,0,1446,3,1,9,2,19,30,4,4,2,0,0,0,3.17,16,7,2,0,3,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.5,0.0,0.0
1,1,1446,3,1,9,2,20,30,4,4,2,0,0,0,3.17,17,7,2,0,3,2.0,2.0,0.0,0.0,0.0,1.0,0.5,0.5,0.0,0.0
2,3,1446,3,1,9,2,21,30,4,4,2,0,0,0,3.17,18,7,2,0,3,0.0,0.0,0.0,1.0,2.0,1.0,0.5,0.0,0.0,0.0
3,0,1458,3,1,9,2,19,30,4,4,2,0,0,0,7.33,16,7,2,0,3,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
4,0,1458,3,1,9,2,20,30,4,4,2,0,0,0,7.33,17,7,2,0,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45396809,0,1434,2,0,9,2,1913,30,4,4,2,0,0,0,3.98,24,16,4,5,4,1.0,0.0,0.0,6.0,0.0,0.0,0.0,1.0,1.0,1.0
45396810,1,1435,2,0,9,2,1912,30,4,4,2,0,0,0,1.28,23,16,4,5,4,0.0,0.0,0.0,1.0,1.0,1.0,1.5,1.0,1.0,1.0
45396811,3,1435,2,0,9,2,1913,30,4,4,2,0,0,0,1.28,24,16,4,5,4,1.0,0.0,4.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0
45396812,0,1436,2,0,9,2,1912,30,4,4,2,0,0,0,1.00,23,16,4,5,4,0.0,1.0,4.0,3.0,2.0,0.0,1.0,2.5,2.0,2.0


In [4]:
#val =  sales_train_evaluation_lag_rolling[(sales_train_evaluation_lag_rolling['d']>=1914) & (sales_train_evaluation_lag_rolling['d']<=1941)]
val = pd.read_csv('s3://{}/{}'.format(bucket, 'data/val.csv'))

In [7]:
val

,demand,item_id,dept_id,cat_id,store_id,state_id,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,date_day,date_week,date_month,date_year,date_week_m,lag_28,lag_35,lag_42,lag_49,lag_56,rolling_median_7,rolling_median_14,rolling_median_30,rolling_median_60,rolling_median_180
45335836,0,1437,3,1,0,0,1914,30,4,4,2,0,0,0,8.38,25,17,4,5,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
45335837,0,1437,3,1,0,0,1915,30,4,4,2,0,0,0,8.38,26,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
45335838,0,1437,3,1,0,0,1916,30,4,4,2,0,0,0,8.38,27,17,4,5,4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
45335839,2,1437,3,1,0,0,1917,30,4,4,2,0,0,0,8.38,28,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
45335840,0,1437,3,1,0,0,1918,30,4,4,2,0,0,0,8.38,29,17,4,5,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46220039,1,1432,2,0,9,2,1940,30,4,4,2,0,0,0,2.98,21,20,5,5,3,0.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0
46220040,1,1433,2,0,9,2,1940,30,4,4,2,0,0,0,2.48,21,20,5,5,3,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
46220041,0,1434,2,0,9,2,1940,30,4,4,2,0,0,0,3.98,21,20,5,5,3,5.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0
46220042,1,1435,2,0,9,2,1940,30,4,4,2,0,0,0,1.28,21,20,5,5,3,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0


In [5]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost', '1.0-1')
container 


The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3'

In [7]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}'.format(bucket, 'data/train.csv'), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}'.format(bucket, 'data/val.csv'), content_type='csv')
(vars(s3_input_train), vars(s3_input_validation))

({'config': {'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
     'S3Uri': 's3://sagemaker-us-west-2-479862333671/data/train.csv',
     'S3DataDistributionType': 'FullyReplicated'}},
   'ContentType': 'csv'}},
 {'config': {'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
     'S3Uri': 's3://sagemaker-us-west-2-479862333671/data/val.csv',
     'S3DataDistributionType': 'FullyReplicated'}},
   'ContentType': 'csv'}})

In [ ]:

prefix = 'perdictions'

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.2xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sagemaker_session)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='reg:linear',
                        num_round=20)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2022-01-10 08:59:02 Starting - Starting the training job...
2022-01-10 08:59:04 Starting - Launching requested ML instancesProfilerReport-1641805142: InProgress
......
2022-01-10 09:00:24 Starting - Preparing the instances for training.........
2022-01-10 09:01:46 Downloading - Downloading input data...............
2022-01-10 09:04:25 Training - Downloading the training image..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','

2022-01-10 09:04:45 Training - Training image download completed. Training in progress.[09:05:39] 4539681

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.2xlarge')

In [ ]:
# We need to tell the endpoint what format the data we are sending is in
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

Y_pred = xgb_predictor.predict(X_test.values).decode('utf-8')
# predictions is currently a comma delimited string and so we would like to break it up
# as a numpy array.
Y_pred = np.fromstring(Y_pred, sep=',')


In [ ]:
for d in range(1914, 1942):
    val['pred_' + str(d)] = xgb.predict(val[val['d']==d])